In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType
from pyspark.sql.functions import trim, col

##Reading From Bronze

In [0]:
df = spark.table("workspace.bronze.crm_cust_info_raw")

In [0]:
df.display()

# Silver Transformations

In [0]:
# Trimming the values
for file in INGESTION: 
    for field in df.schema.fields:
        if isinstance(field.dataType, StringType):
            df=df.withColumn(field.name, trim(col(field.name)))

In [0]:
# Normalizations: spell out marital status and gender
df = ( 
      df
      .withColumn(
          "cst_marital_status", 
          F.when(F.upper(F.col("cst_marital_status")) == "S", "Single")
          .when(F.upper(F.col("cst_marital_status")) == "M", "Married") 
          .otherwise("n/a")
      )
      .withColumn(
          "cst_gndr",
          F.when(F.upper(F.col("cst_gndr")) == "M", "Male") 
          .when(F.upper(F.col("cst_gndr")) == "F", "Female")
          .otherwise("n/a")
      )
)

##Removing Records with Missing Customer ID


In [0]:
df = df.filter(col("cst_id").isNotNull()) # remove nulls

##Renaming Columns

In [0]:
RENAME_MAP = {
    "cst_id": "customer_id",
    "cst_key": "customer_number",
    "cst_firstname": "firstname",
    "cst_lastname": "lastname",
    "cst_marital_status": "marital_status",
    "cst_gndr": "gender",
    "cst_create_date": "created_date"
}
for old_name, new_name in RENAME_MAP.items(): 
    df = df.withColumnRenamed(old_name, new_name)

### Data Quality Checks of Dataframe

In [0]:
df.display()

##Write Into Silver Table

In [0]:
df.write.mode("overwrite").format("delta").saveAsTable("workspace.silver.crm_customers")

## DQ checks of silver table

In [0]:
%sql 
SELECT * FROM workspace.silver.crm_customers LIMIT 10